In [ ]:
import cv2
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
import numpy as np
#加载模型

model = load_model('face_96.h5')


def face_predict(image): 
    
    image = np.resize(image,(96,96,3))    
    x = img_to_array(image)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
     
    #给出类别预测：0或者1
    result = model.predict(x)
    result=np.argmax(result,axis=1)
    #print('result:', result)   

    #返回类别预测结果
    return result


#框住人脸的矩形边框颜色       
color = (0, 255, 0)
    
#捕获指定摄像头的实时视频流
cap = cv2.VideoCapture(0)
    
#人脸识别分类器本地存储路径
cascade_path = "/usr/local/share/OpenCV/haarcascades/haarcascade_frontalface_alt2.xml"    
    
#循环检测识别人脸
while True:
    _, frame = cap.read()   #读取一帧视频
        
    #图像灰化，降低计算复杂度
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
    #使用人脸识别分类器，读入分类器
    cascade = cv2.CascadeClassifier(cascade_path)                

    #利用分类器识别出哪个区域为人脸
    faceRects = cascade.detectMultiScale(frame_gray, scaleFactor = 1.2, minNeighbors = 3, minSize = (32, 32))        
    if len(faceRects) > 0:                 
        for faceRect in faceRects: 
            x, y, w, h = faceRect
                
            #截取脸部图像提交给模型识别这是谁
            image = frame[y - 10: y + h + 10, x - 10: x + w + 10]
            faceID = face_predict(image)   
                
            #如果是“我”
            if faceID == 0:                                                        
                cv2.rectangle(frame, (x - 10, y - 10), (x + w + 10, y + h + 10), color, thickness = 2)
                    
                #文字提示是谁
                cv2.putText(frame,'yangbinfeng', 
                            (x + 30, y + 30),                      #坐标
                            cv2.FONT_HERSHEY_SIMPLEX,              #字体
                            1,                                     #字号
                            (255,0,255),                           #颜色
                            2)                                     #字的线宽
            elif faceID == 1: 
                cv2.rectangle(frame, (x - 10, y - 10), (x + w + 10, y + h + 10), color, thickness = 2)
                    
                #文字提示是谁
                cv2.putText(frame,'guanmeie', 
                            (x + 30, y + 30),                      #坐标
                            cv2.FONT_HERSHEY_SIMPLEX,              #字体
                            1,                                     #字号
                            (255,0,255),                           #颜色
                            2)                                     #字的线宽
            else:
                pass
                            
    cv2.imshow("识别朕", frame)
        
    #等待10毫秒看是否有按键输入
    k = cv2.waitKey(10)
    #如果输入q则退出循环
    if k & 0xFF == ord('q'):
        break

#释放摄像头并销毁所有窗口
cap.release()
cv2.destroyAllWindows()